한국어 자연어처리의 전반적인 FLOW를 이해하고, 간단한 감성분석 모델을 구현해 봅시다

필요한 라이브러리를 import해줍니다.

LSTM을 활용해 RNN 신경망을 구성하고, Embedding을 통해 단어를 벡터로 숫자로 바꿔주겠습니다.








In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM 

📋 교재로 돌아가 한국어 자연어처리 플로우를 보고 이해한 뒤 실습을 시작합시다!

### Step 1. Tokenizing(Parsing)
- 문장을 음절(character)단위로 쪼개기


In [ ]:
def tokenize(sentence): 
  return [char for char in sentence]

In [ ]:
sentence1 = "시간 가는 줄 알고 봤습니다."
sentence2 = "안보면 후회ㅠㅠ..."
parsed_sent1 = tokenize(sentence1)
parsed_sent2 = tokenize(sentence2)
print("문장 1:", parsed_sent1)
print("문장 2:", parsed_sent2)

### Step 2. 모델 인풋 만들기


#### 2-1) 음절 사전 만들기
각 음절 캐릭터를 모델이 처리할 수 있는 정수 인덱스로 변환해야 합니다.
- 캐릭터를 정수로 매핑하는 사전을 만들고,
- 배치 연산을 위해 필요한 Padding([PAD])과 Out of vocabulary([OOV]) 토큰을 항상 맨 앞에 추가해줍니다

In [ ]:
vocab_dict = {}
vocab_dict["[PAD]"] = 0
vocab_dict["[OOV]"] = 1
i = 2
for word in parsed_sent1:
    if word not in vocab_dict.keys():
        vocab_dict[word] = i
        i += 1
for word in parsed_sent2:
    if word not in vocab_dict.keys():
        vocab_dict[word] = i
        i += 1
print("Vocab Dictionary Example:")
print(vocab_dict)

#### 2-2) vocab_dict를 이용해 자연어를 정수 인덱스로 바꾸기
- 위에서 만든 vocab_dict를 이용해 잘라놓은 문장을 모델에 태울 수 있는 정수 인덱스로 바꾸어줍니다
- 기본적으로 LSTM은 가변적인 문장 길이를 인풋으로 받을 수 있지만, 배치 처리를 위해 <font color="blue">max_seq_len</font>을 정해두고 길이를 통일합니다.
    - max_seq_len 보다 짧은 문장에는 max_seq_len이 될 때까지 [PAD]에 해당하는 인덱스를 붙여줍니다
    - max_seq_len 보다 긴 문장은 max_seq_len 개의 토큰만 남기고 자릅니다
    - tensorflow.keras.preprocessing.sequence의 <font color="blue">pad_sequences</font> 기능을 사용합니다.

In [ ]:
max_seq_len = 20

input_id1 = [vocab_dict[word] for word in parsed_sent1]
input_id2 = [vocab_dict[word] for word in parsed_sent2]

# Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_ids = [input_id1, input_id2]
input_ids = pad_sequences(input_ids, maxlen=max_seq_len, value = vocab_dict['[PAD]']) 
print(input_ids)

### Step3. 모델 만들기

- 임베딩 레이어 : Embedding()
- LSTM : LSTM()
- FC layer : Dense()   
- LSTM을 사용해 문장을 인코딩하고, Dense layer을 두 층 쌓아 최종 output을 생성합시다

In [ ]:
vocab_size = len(vocab_dict)        # 단어사전 개수
embedding_dim = 30     # 임베딩 차원
lstm_hidden_dim = 50   # LSTM hidden_size 
dense_dim = 32         # FC layer size
batch_size = 2         # batch size

model = Sequential([
    Embedding(vocab_size, embedding_dim),
    LSTM(lstm_hidden_dim),
    Dense(dense_dim, activation='relu'),
    Dense(2, activation='softmax')
])

model.summary()

# LSTM으로 영화리뷰 감성분석 모델 훈련하기

이제 본격적으로 영화리뷰를 다운받고 실습을 진행합니다.

### Step 0. 학습 데이터 준비하기
<img src = "https://github.com/seungyounglim/temporary/blob/master/image_5.PNG?raw=true">    

- 네이버 영화 감성분석 데이터셋 활용
- 훈련 데이터 150,000건, 테스트 데이터 50,000건

In [ ]:
""" 네이버 영화 리뷰 데이터셋 다운로드 """
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

""" 데이터 읽어오기 """
with open("ratings_train.txt") as f:
    raw_train = f.readlines()
with open("ratings_test.txt") as f:
    raw_test = f.readlines()
raw_train = [t.split('\t') for t in raw_train[1:]]
raw_test = [t.split('\t') for t in raw_test[1:]]

FULL_TRAIN = []
for line in raw_train:
    FULL_TRAIN.append([line[0], line[1], int(line[2].strip())])
FULL_TEST = []
for line in raw_test:
    FULL_TEST.append([line[0], line[1], int(line[2].strip())]) 

<img src = "https://github.com/seungyounglim/temporary/blob/master/image_6.PNG?raw=true">  
- 시간 관계상 train 중 50,000건을 학습데이터, 10,000건을 검증 데이터로 사용합니다.
- test 중 10,000건만 샘플링하여 최종 성능 테스트에 사용하겠습니다

In [ ]:
import random
random.seed(1)
random.shuffle(FULL_TRAIN)
random.shuffle(FULL_TEST)
train = FULL_TRAIN[:50000]
val = FULL_TRAIN[50000:60000]
test = FULL_TEST[:10000]
print("train     : {}개 (긍정 {}, 부정 {})".format(len(train), sum([t[2] for t in train]), len(train)-sum([t[2] for t in train])), train[0])
print("validation: {}개 (긍정 {}, 부정 {})".format(len(val), sum([t[2] for t in val]), len(val)-sum([t[2] for t in val])), val[0])
print("test      : {}개 (긍정 {}, 부정 {})".format(len(test), sum([t[2] for t in test]), len(test)-sum([t[2] for t in test])), test[0])

라벨을 보니 0이 부정 리뷰, 1이 긍정 리뷰입니다. 

한국말은 끝까지 읽어봐야 합니다.

## Step 1. Parsing
- Train/ Test의 문장을 음절단위로 파싱합니다
- 정답 라벨은 One-hot encoding 형식으로 저장합니다.
   - 부정(0) -> [1, 0]
   - 긍정(1) -> [0, 1]

In [ ]:
train_sentences = []
val_sentences = []
test_sentences = []

# 추후 학습/ 테스트를 위해 라벨 정보 저장해둠
train_label_ids = []
val_label_ids = []
test_label_ids = []

print("start tokenizing TRAIN sentences")
for i, line in enumerate(train):
    words = tokenize(line[1])
    train_sentences.append(words)
    if line[2] == 0: # 부정
      train_label_ids.append([1,0])
    else: #긍정
      train_label_ids.append([0,1])

    if (i+1) % 5000 == 0: print("... {}/{} done".format(i+1, len(train))) 

print("start tokenizing VALIDATION sentences")

for line in val:
    words = tokenize(line[1])
    val_sentences.append(words)
    if line[2] == 0: # 부정
      val_label_ids.append([1,0])
    else: #긍정
      val_label_ids.append([0,1])
print("... done\n")

print("start tokenizing TEST sentences")
for line in test:
    words = tokenize(line[1])
    test_sentences.append(words)
    if line[2] == 0: # 부정
      test_label_ids.append([1,0])
    else: #긍정
      test_label_ids.append([0,1])

print("... done")

##Step 2. 모델 인풋 만들기

#### 2-1) 음절 사전 만들기
- 훈련 데이터 문장에 있는 음절을 이용해 사전을 만듭니다.
- (일반적으로는 더 많은 코퍼스에 대해 구축된 사전을 사용하지만, 편의상 훈련셋만으로 진행합니다)
 

In [ ]:
from tqdm import tqdm

vocab_dict = {}
vocab_dict["[PAD]"] = 0
vocab_dict["[OOV]"] = 1
i = 2
for sentence in train_sentences:
    for word in sentence:
        if word not in vocab_dict.keys(): 
            vocab_dict[word] = i
            i += 1
print("Vocab Dictionary Size:", len(vocab_dict))

#### 2-2) vocab_dict를 이용해 자연어를 정수 인덱스로 바꾸기



음절 단위로 쪼개진 문장들 (tokenized_sentences)을 인풋으로 받아 다음을 처리합니다.

* 사전에 없는 음절은 [OOV] 인덱스로 처리하기   
* 사전에서 매핑되는 음절은 해당 인덱스로 바꾸기
* max_seq_len만큼 문장 길이 맞추고 이보다 짧은 문장은 [PAD] 인덱스로 패딩하기


In [ ]:
def make_input_ids(tokenized_sentences, max_seq_len = 50):
  
  num_oov = 0 # OOV 발생 개수를 셈
  result_input_ids = [] # result_input_ids : 정수 인덱스로 변환한 문장들의 리스트

  for sentence in tokenized_sentences :
      """ vocab_dict를 사용해 정수로 변환 """ 
      input_ids = []
      for word in sentence:
          if word not in vocab_dict:   ## 사전에 없는 음절은 OOV 처리
              input_ids.append(vocab_dict['[OOV]']) 
              num_oov += 1
          else:                       ## 사전에 있는 음절은?
              input_ids.append(vocab_dict[word]) ##  vocab_dict 사전에서 토큰 찾아서 붙이기
      
      result_input_ids.append(input_ids)
      
  """ max_seq_len을 넘는 문장은 절단, 모자르는 것은 PADDING """
  result_input_ids = pad_sequences(result_input_ids, maxlen=max_seq_len, value=vocab_dict["[PAD]"]) ##  padding 하기

  return result_input_ids, num_oov


In [ ]:
# train_sentences 처리
train_input_ids, num_oov = make_input_ids(train_sentences)

print("---- TRAIN ----")
print("... # OOVs     :", num_oov)

In [ ]:
# val_sentences 처리
val_input_ids, num_oov = make_input_ids(val_sentences)

print("---- VALIDATION ----")
print("... # OOVs     :", num_oov)

In [ ]:
# test_sentences 처리
test_input_ids, num_oov = make_input_ids(test_sentences)

print("---- TEST ----")
print("... # OOVs     :", num_oov)

#### 2-3) 라벨 리스트를 np.array로 변환해줍니다.


In [ ]:
train_label_ids = np.array(train_label_ids)
val_label_ids = np.array(val_label_ids)
test_label_ids = np.array(test_label_ids)

## Step3. 모델 만들기

## 실습 MISSION
> 아래 조건에 맞는 모델을 만들어보고 학습하세요
 
* embedding 차원은 150  
* LSTM hidden size는 100
* Dense의 hidden size는 100, relu activation 사용
* output Dense layer에서는 긍/부정 2개 카테고리를 분류하되 softmax 사용

In [ ]:
vocab_size = len(vocab_dict) 

model = Sequential([
            ####### MISSION 작성 ######
            Embedding(vocab_size, 150),
            LSTM(100),
            Dense(100, activation='relu'),
            Dense(2, activation='softmax')
            ###########################
])  

model.summary()

## Step 4. 모델 훈련하기

#### 4-1) loss, optimizer를 지정하고 학습합니다.


In [ ]:
EPOCHS = 10
BATCHS = 256

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_input_ids, train_label_ids, epochs=EPOCHS, batch_size=BATCHS, validation_data=(val_input_ids, val_label_ids), verbose=2) 

test_result = model.evaluate(test_input_ids, test_label_ids, verbose=2)

#### 4-2) Learning Curve 확인하기

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

## Step 5. Inference 실행하기

In [ ]:
""" 학습된 모델로  예측해보기 """

def inference(mymodel, sentence):
  # 1. tokenizer로 문장 파싱
  words = tokenize(sentence)
  input_id = []

  # 2. vocab_dict를 이용해 인덱스로 변환
  for word in words:
    if word in vocab_dict: input_id.append(vocab_dict[word])
    else: input_id.append(vocab_dict["[OOV]"])
  
  # 단일 문장 추론이기 때문에 패딩할 필요가 없음 
  score = mymodel.predict(np.array([input_id])) 

  print("** INPUT:", sentence)
  print("   -> 부정: {:.2f} / 긍정: {:.2f}".format(score[0][0],score[0][1]))

In [ ]:
sentence1 = "시간 가는 줄 알고 봤습니다."
sentence2 = "안보면 후회ㅠㅠ..."
inference(model, sentence1)
inference(model, sentence2)

In [ ]:
# 원하는 문장에 대해 추론해 보세요
inference(model, "이런 망작을 나 혼자만 보기엔 아깝지")
inference(model, "이런 꿀잼을 나 혼자만 보기엔 아깝지")
inference(model, "꿀잠 잤습니다")

# # 3. 나만의 모델 만들어보기 

# 실습 MISSION
>  LSTM, Dense layer 등을 자유롭게 활용해서 자신만의 모델을 만들고 
이후 TEST 데이터에 대해 최종 성능을 비교해보세요
</font>
 

In [ ]:
tf.keras.backend.clear_session()
 

# 1. 모델 구현하기
model2 = tf.keras.Sequential([
      # MISSION 작성 #


      ################                 
]) 

# 2. optimizer, loss 선택하기
model2.compile(loss='_______________', optimizer='________', metrics=['accuracy'])

# 3. 모델 훈련하기 (원하는대로 조정해보세요!)
num_epochs = 5
num_batch = 256

history = model2.fit(train_input_ids, train_label_ids, epochs=num_epochs, batch_size=num_batch, validation_data=(val_input_ids, val_label_ids), verbose=2)

In [ ]:
# 4. 모델 진단하기

plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
# 5. 테스트 데이터에 대해 평가하기

model2.evaluate(test_input_ids, test_label_ids, verbose=2)

In [ ]:
# 샘플 예제에 대해 추론해 보세요 

inference(model2, "물이 반도 안남았다")  #부정
inference(model2, "물이 반이나 남았다")  #긍정
inference(model2, "죄송하지만 혹시 실례가 안된다면 꺼져주실수 있으신지ㅎㅎ?") #부정
inference(model2, "잘하는 짓이다") #부정
inference(model2, "가게 외관은 구린데 맛은 ㅇㅈ") #긍정
inference(model2, "ㄷㄷ 간만에 갓띵작 ㄷㄷㄷ") #긍정
inference(model2, "주인공 커여워 ㅠㅠ") #긍정
inference(model2, "OTL") #부정
